# Transform image of array to digital values 


This notebook provides:
        - search for spots on array
        - mapping images postions peptides to peptides informations
        - extracting intensities of spots and mapping to coresponding spots
        - writing gal-file for further analysis with analysis script 
        


In [1]:
import numpy as np
import pandas as pd
import math
import cv2
import argparse
import imutils
import os

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as patches
import matplotlib.colors as colors
import matplotlib.image as mpimg

from sklearn import decomposition
from sklearn.preprocessing import normalize
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import NearestNeighbors



from skimage import data, io, filters,measure
from skimage.feature import match_template 

import scipy
from scipy.ndimage.interpolation import shift
from scipy import ndimage,spatial
from scipy.optimize import leastsq, minimize
import scipy.ndimage.filters as filters



from IPython.display import display, HTML
from PIL import Image
import PIL.ImageOps 
from imutils import contours

from  flutype_analysis import image2numeric, utils



## define help functions

## Load data
    -Load galfile of peptides
    -Load images after spotting but before incubation with virus
    -Load images after incubation with virus

In [2]:

data_ids = {'N10'}
#--------------------------
#data_id = 'E5'
data_id = 'N10'
#--------------------------
if any(data_id == d_id for d_id in ['N10', 'E5' ]):
    #load raw data
    
    #pep= pd.read_csv("data/{}_pep.gal".format(data_id), sep='\t',index_col="ID")
    for fname in os.listdir('data/'):    # change directory as needed
        if os.path.isfile('data/{}'.format(fname)):# make sure it's a file, not a directory entry
            #print(fname)
            if "{}_600_100_635.tif".format(data_id) in fname:    # search for string
                print(fname)
                
                im =  cv2.imreadmulti("data/{}".format(fname), 0)
            elif "{}_after_600_100_635.tif".format(data_id) in fname:    # search for string
                print(fname)
                imafter =  cv2.imreadmulti("data/{}".format(fname), 0)
            elif "{}_pep.gal".format(data_id) in fname:
                print(fname)
                pep = pd.read_csv("data/{}".format(fname), sep='\t',index_col="ID")
                pep_cor = pep.pivot(index="Row", columns="Column", values="Name")    
# unstack peptide coordinates data
pep_cor_unstacked=pep_cor.unstack()
# only fluorecent data
leuchtefix=pep_cor_unstacked.str.contains("Leuchte")


    


NameError: name 'pep_cor' is not defined

In [ ]:
def estimate_grid(points,Peptides):
    # estimated pitch on image
    pitch=(points[:,0].max()-points[:,0].min())/(Peptides["Column"].iloc[nonzero_ids].max()-Peptides["Column"].iloc[nonzero_ids].min())
    #estimated center x on image
    center_x=points[:,0].min()+0.5*(points[:,0].max()-points[:,0].min())
    #estimated center y on image
    center_y=points[:,1].min()+0.5*(points[:,1].max()-points[:,1].min())
    #gridshape
    gridshape=(Peptides["Row"].max(),Peptides["Column"].max())
    return pitch, center_x, center_y, gridshape

In [1]:
data_id="2017-05-19_E5_X31" 
directory = os.path.join("../data",data_id)
data = utils.load_data(data_id,directory,what="image2numeric")
ana=analysis.Analysis(data)


NameError: name 'os' is not defined

In [ ]:

draw_imag(im,"before incubation")
draw_imag(imafter,"after incubation")

# Define functions for Spot detection:

In [ ]:
circles=detect_circles(im)
draw_circles(im,circles)

## Maps spot positions in image to peptide in gal file



In [ ]:
# create a new Dataframe with x and y coordinates of peptides
Peptides=pd.DataFrame(pep_cor_unstacked.values,columns=["Peptides"])
Peptides["Column"]=leuchtefix.index.get_level_values(0).values
Peptides["Row"]=leuchtefix.index.get_level_values(1).values
Peptides["x"]=0
Peptides["y"]=0

display(Peptides.head())

# generate a new data frame with x and y coordinates of spots detected by previous algorithm
spot_position = pd.DataFrame(circles[0,:,0:2], columns=["x","y"])
display(spot_position)

# display all spots containing Leuchtefix
nonzero_ids=leuchtefix.nonzero() # ids of spots in gal file containing Leuchtefix
display(Peptides.iloc[nonzero_ids])

### find grid on image & map leuchtefix position on image to galfile 


In [ ]:

# estimated pitch on image
pitch=(spot_position["x"].max()-spot_position["x"].min())/(Peptides["Column"].iloc[nonzero_ids].max()-Peptides["Column"].iloc[nonzero_ids].min())
#estimated center x on image
center_x=spot_position["x"].min()+0.5*(spot_position["x"].max()-spot_position["x"].min())
#estimated center y on image 
center_y=spot_position["y"].min()+0.5*(spot_position["y"].max()-spot_position["y"].min())
#gridshape
gridshape=(Peptides["Row"].max(),Peptides["Column"].max())





#optimzes grid parameters by minimizing error function:
best_arg=minimize(err_func, x0=(pitch, center_x, center_y, 0), args=(spot_position["x"].values,spot_position["y"].values,gridshape))
X,Y=get_grid(gridshape,pitch=best_arg.x[0],center_x=best_arg.x[1],center_y=best_arg.x[2],rotation=best_arg.x[3])

#draw the grid on image
draw_grid(im.T,Y,X,"optimzed Grid")


## Map leuchtefix positions to gal file

In [ ]:
_, _ , leuchtefix_indexes_on_grid=get_nearest_on_grid(X,Y,spot_position["x"].values,spot_position["y"].values)

for index,grid_index in enumerate(leuchtefix_indexes_on_grid):
    Peptides['x'].iloc[grid_index]=spot_position["x"][index]
    Peptides['y'].iloc[grid_index]=spot_position["y"][index]
    
display(Peptides.iloc[nonzero_ids])  
draw_grid(-im.T,Peptides["y"].iloc[nonzero_ids],Peptides["x"].iloc[nonzero_ids],"check if maping Leuchtefix to gal file worked:")



#draw_imag(inverte(imafter.T),"inverted image", cmap="gray")

gray = imafter.T

kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(gray,cv2.MORPH_OPEN,kernel, iterations = 2)

draw_imag(gray,"inverted",cmap="gray")
draw_imag(opening,"noise removal",cmap="gray")
# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3)
draw_imag(sure_bg,"noise removal",cmap="gray")
# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
draw_imag(sure_fg,"noise removal",cmap="gray")
#unknown = cv2.subtract(sure_bg,sure_fg)

ret, markers = cv2.connectedComponents(sure_bg)

# Add one to all labels so that sure background is not 0, but 1
draw_imag(markers,"markers")


In [ ]:
# show ratio of shape of images:
ratio = (im.shape[0]/imafter.shape[0],im.shape[1]/imafter.shape[1])
print(ratio)

points_x = []
points_y = []
for marker in range(markers.max()):
    point= np.array(np.where(markers == marker))
    points_x.append(point[0].mean())
    points_y.append(point[1].mean())

    
#draw_grid(imafter.T,points_y,points_x,"scatter")



xy_grid=np.vstack([X/ratio[0]+3,Y/ratio[0]-25]).reshape(2,-1).T
Peptides['x'] = xy_grid[:,0]
Peptides['y'] = xy_grid[:,1]
blank=pep_cor_unstacked.str.contains("blank")
blank_ids=blank.nonzero()
Peptides['x'].iloc[blank_ids] = float('nan')
Peptides['y'].iloc[blank_ids] = float('nan')



    
new_points =[]
pts=np.vstack([points_x,points_y]).reshape(2,-1).T
T = spatial.KDTree(xy_grid)
idx = T.query_ball_point(pts,r=50)
points_new= []
#print(len(idx))
#print(len(pts))
for index,i in enumerate(idx):
    if len(i) > 0:
        points_new.append(pts[index])
points_new = np.array(points_new)
#print(len(points_new))
#print(Peptides['x'].value_counts(dropna=False))
fig, ax = plt.subplots(figsize=(30,10))
plt.scatter(Peptides['y'],Peptides['x'])
plt.scatter(points_new[:,1],points_new[:,0])
plt.imshow(markers)
plt.show()




In [ ]:
import matplotlib.colors as colors
import matplotlib.cm as cmx

finite = np.isfinite(Peptides['x'])
pts=np.vstack([Peptides[finite]['x'],Peptides[finite]['y']]).reshape(2,-1).T
print(len(points_new))
nearest = nearest_neighbour(points_new, pts)
print(len(nearest))


jet = cm = plt.get_cmap('flag') 
cNorm  = colors.Normalize(vmin=0, vmax=len(nearest))
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)



fig, ax = plt.subplots(figsize=(30,10))
for i_points_new,i in enumerate(nearest):
    colorVal = scalarMap.to_rgba(i_points_new)
    plt.scatter(pts[i,1],pts[i,0],color=colorVal)
    plt.scatter(points_new[i_points_new,1],points_new[i_points_new,0], color=colorVal)
plt.imshow(markers)


plt.show()



In [ ]:
pitch, center_x, center_y, gridshape = estimate_grid(points_new,Peptides)
best_arg=minimize(err_func, x0=(pitch, center_x, center_y, 0), args=(points_new[:,0],points_new[:,1],gridshape))
X,Y=get_grid(gridshape,pitch=best_arg.x[0],center_x=best_arg.x[1],center_y=best_arg.x[2],rotation=best_arg.x[3])
pts=np.vstack([X,Y]).reshape(2,-1).T


print(finite.nonzero())
#draw the grid on image
fig, ax = plt.subplots(figsize=(30,10))

plt.scatter(points_new[:,1],points_new[:,0])
plt.scatter(pts[finite.nonzero(),1],pts[finite.nonzero(),0],marker="+")
plt.imshow(markers)
plt.show()




In [ ]:
fig2 = plt.figure(figsize=(30,10))
ax2 = fig2.add_subplot(111, aspect='equal')

for p in [
    patches.Rectangle(
        (patchcenter[1]-0.5*pitch, patchcenter[0]-0.5*pitch),
        pitch,
        pitch,
        fill=False,      # remove background
        linewidth=1,
        edgecolor='r'
    ) for patchcenter in pts[finite.nonzero()]
]:
    ax2.add_patch(p)
plt.imshow(markers)
plt.show()

In [ ]:


#display(Peptides)

#draw_grid(markers, Peptides[finite][~nonzero_ids]["y"],Peptides[finite][~nonzero_ids]["x"],"test")

Peptides["x"].iloc[Peptides[~leuchtefix.values][finite].index] =pts[Peptides[~leuchtefix.values][finite].index][:,0]
Peptides["y"].iloc[Peptides[~leuchtefix.values][finite].index]=pts[Peptides[~leuchtefix.values][finite].index][:,1]

#draw_grid(markers, Peptides[finite][~nonzero_ids]["y"],Peptides[finite][~nonzero_ids]["x"],"test")
draw_grid(markers, Peptides["y"],Peptides["x"],"test")




In [ ]:
fig2 = plt.figure(figsize=(30,10))
ax2 = fig2.add_subplot(111, aspect='equal')

for p in [
    patches.Rectangle(
        (patchcenter_y-0.5*pitch, patchcenter_x-0.5*pitch),
        pitch,
        pitch,
        fill=False,      # remove background
        linewidth=1,
        edgecolor='r'
    ) for patchcenter_x, patchcenter_y in zip(Peptides[finite]["x"],Peptides[finite]["y"] )]:
    ax2.add_patch(p)
plt.imshow(markers)
plt.show()

## change grid position by hand to match image after incubation:

In [ ]:


if ratio[0] == ratio[1]:
    #scale position of spots:
    circles_new = circles/ratio[0]
    #changes position of spots
    circles_new[0,:,1] = circles_new[0,:,1]-25
    circles_new[0,:,0] = circles_new[0,:,0]+3
    
    Y_after=  Y/ratio[0]-25

    draw_grid(markers,Y/ratio[0]-25,X/ratio[0]+3,"check alignment of pictures")
    draw_imag(im.T,"inverted", cmap="gray")
    draw_imag(imafter.T,"orginal", cmap="gray")
    

    display(pep_cor.T,"small")
else:
    print("sizes of images before incubation and after incubation do not match (not possible to scale x and y axis with same factor" )


# Problems:
- different resolutions for different images.(todo: standard protokol peptide arrays, same resolution, same gain, same oriantation)
- different intensities before and after washing.
- not a regular grid (spotter? solution maybe : industial


# Questions:
- what is the spotting order?
- AK klumpt und fluoreszenz istvorhanden (bevor virus). Ist Antikörper schon mit Virus verschmutz?
- Wie AK an Mikroarray befestigt?

In [ ]:
display(Peptides)

## calculate spot intensities


In [ ]:
Peptides["Intensity"]=0

for index in Peptides.index:

    if not math.isnan(Peptides.loc[index]['x']):
        x_min = int(Peptides.loc[index]['x']-0.5*pitch)
        x_max = int(Peptides.loc[index]['x']+0.5*pitch)
        y_min = int(Peptides.loc[index]['y']-0.5*pitch)
        y_max = int(Peptides.loc[index]['y']+0.5*pitch)
        intensity=imafter[y_min:y_max,x_min:x_max].sum()
        Peptides.set_value(index,"Intensity",intensity)
        
   

    

## generate csv file as readeroutput for intensity analysis

In [ ]:
output=Peptides.pivot(index="Row", columns="Column", values="Intensity")
output.to_csv('data/{}.csv'.format(data_id), sep='\t')